### Workflow:

- Before start:
    - Add files
    - Combine all files
    - import libraries
- Experiment:
    - EVERYTHING

In [ ]:
print("Help! Please\n :')")

Help! Please
 :')


In [6]:
import os

def list_files_in_current_directory():
    # Get the current directory
    current_directory = os.getcwd()

    # List all files in the current directory
    files = os.listdir(current_directory)

    # Print each file
    for file in files:
        # Check if the path is a file (not a directory)
        if os.path.isfile(os.path.join(current_directory, file)):
            print(file)

# Call the function to list files in the current directory
list_files_in_current_directory()
